# **Personal Information**

### $Name-Surname:$ $Eren$ $Kotar$
### $e-mail: erenkotar@hotmail.com$

# **NOTE**

**Since I could not find any data science application form in any of INSIDER's job announcements I thought that the data science department is a sub-department of the software engineering department.  But considering the online internship event and the task given, it is not :). I have no knowledge of the front-end. Thus, I could not finish the task properly. But since Morte Carlo simulation is related to statistics, I took that assignment as a kind of data science task and did my project in that fashion.**

**In the project, I scraped the 20-year statistics of the  Premier League and created my random variates according to those statistics. Since form (recency) is important in soccer, I created my statistics only from the 2013-2014's season.**

**I have created my random variates by combining the goal distribution of that specific team against all teams and the goal distribution of that specific team agaisnt that specific team. The weight of the first (general) distribution is 0.4 and the second (team-specific) distribution is 0.6.**

## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from pprint import pprint as pp

## Importing Premier League dataset

In [2]:
df = pd.read_csv("final_dataset.csv",index_col=0,parse_dates=["Date"])
df = df[df.Date >= "2013-07-19"] # taking data from 2013-2014's season to now

teams = ["Chelsea","Arsenal","Man City","Liverpool"] # teams will be simulated 

In [3]:
class Team:
    def __init__(self, name_of_club):
        self.name = name_of_club
    
    # Calculates general home goal distribution of team object agaisnt all other clubs
    # Example: Chelsea agaisnt all clubs
    def general_dist_home(self, df:pd.DataFrame):
        ser1 = pd.Series(0,index=np.arange(9),name="main")
        home_goal = "FTHG"
        ser_home = (df.loc[df.HomeTeam==self.name,home_goal].value_counts() / df.loc[df.HomeTeam==self.name,home_goal].value_counts().sum()).sort_index()
        ser_home = pd.merge(ser1,ser_home,how="left",left_index=True,right_index=True)[["FTHG"]].fillna(0)
        return ser_home
    
    # Calculates general away goal distribution of team object agaisnt all other clubs
    def general_dist_away(self, df:pd.DataFrame):
        ser1 = pd.Series(0,index=np.arange(9),name="main")
        away_goal = "FTAG"
        ser_away = (df.loc[df.AwayTeam==self.name,away_goal].value_counts() / df.loc[df.AwayTeam==self.name,away_goal].value_counts().sum()).sort_index()
        ser_away = pd.merge(ser1,ser_away,how="left",left_index=True,right_index=True)[["FTAG"]].fillna(0)
        return ser_away

In [4]:
def create_dict(teams,objects):
    team_object_dict = {i+1:objects[i] for i in range(len(teams))}
    team_dict = {i+1:teams[i] for i in range(len(teams))}
    return team_object_dict,team_dict

# Since all teams will be an object, I have created an dictionary containing object and it's index

In [5]:
def generate_teams(teams):
    for i in range(len(teams)):
        globals()[f"t{i+1}"] = Team(teams[i])
# Generating teams as an object, variable names are: t1, t2.. 

In [6]:
# Calculates specific distribution of home and away goal distribution of team object agaisnt that specific club 
# Example: Chelsea agaisnt Arsenal
def specific_dist(df, home_team, away_team):
    ser = pd.Series(0,index=np.arange(9),name="main")
    
    ser_home = df.loc[(df.HomeTeam==home_team.name) & (df.AwayTeam==away_team.name) ,"FTHG"].value_counts() / df.loc[(df.HomeTeam==home_team.name) & 
                                                                                                                     (df.AwayTeam==away_team.name) ,"FTHG"].value_counts().sum()

    ser_away = df.loc[(df.HomeTeam==home_team.name) & (df.AwayTeam==away_team.name) ,"FTAG"].value_counts() / df.loc[(df.HomeTeam==home_team.name) & 
                                                                                                                     (df.AwayTeam==away_team.name) ,"FTAG"].value_counts().sum()

                         
    ser_home = pd.merge(ser,ser_home,how="left",left_index=True,right_index=True)[["FTHG"]].fillna(0)
    ser_away = pd.merge(ser,ser_away,how="left",left_index=True,right_index=True)[["FTAG"]].fillna(0)
    return ser_home, ser_away

In [7]:
def combined_distribution(df, home_team, away_team, gdw = 0.4, sdw = 0.6):
    # sdw: specific distribution weight, gdw: general distribution weight
    
    home_specific_dist, away_specific_dist = specific_dist(df,home_team,away_team)
    general_home_dist = home_team.general_dist_home(df)
    general_away_dist = away_team.general_dist_away(df)
    
    home_PMF = general_home_dist*gdw + home_specific_dist*sdw
    away_PMF = general_away_dist*gdw + away_specific_dist*sdw
        
    return home_PMF, away_PMF

In [8]:
def match_result(df, home_team, away_team):
    home_PMF, away_PMF = combined_distribution(df, home_team, away_team)
    
    home_goal = np.random.choice(home_PMF.index,p=home_PMF.values.reshape(-1,))
    away_goal = np.random.choice(away_PMF.index,p=away_PMF.values.reshape(-1,))
    
    return home_goal, away_goal
    

In [9]:
# Calculating fixture

def make_day(num_teams, day):
    
    # generate list of teams
    lst = list(range(1, num_teams + 1))
    # rotate
    day %= (num_teams - 1)  # clip to 0 .. num_teams - 2
    if day:                 # if day == 0, no rotation is needed (and using -0 as list index will cause problems)
        lst = lst[:1] + lst[-day:] + lst[1:-day]
    # pair off - zip the first half against the second half reversed
    half = num_teams // 2
    return list(zip(lst[:half], lst[half:][::-1]))

def make_schedule(num_teams):
    
    schedule = [make_day(num_teams, day) for day in range(num_teams - 1)]
    # generate second round-robin by swapping home,away teams
    swapped = [[(away, home) for home, away in day] for day in schedule]

    return schedule + swapped

def fixture(team_dict):
    num_teams = len(team_dict)
    schedule = make_schedule(num_teams)
    fixture_array = np.array(schedule)
    np.random.shuffle(fixture_array)
    
    return fixture_array

In [10]:
def play_week(object_dict, df, fixture_array, week):
    first_match = fixture_array[week][0]
    second_match = fixture_array[week][1]
    
    fm_home_goal, fm_away_goal = match_result(df, object_dict[first_match[0]], object_dict[first_match[1]])
    sm_home_goal, sm_away_goal = match_result(df, object_dict[second_match[0]], object_dict[second_match[1]])
    
    result_list = np.array([[fm_home_goal, fm_away_goal],[sm_home_goal, sm_away_goal]])
    return result_list

In [11]:
def main(teams):
    generate_teams(teams)
    team_object_dict,team_dict = create_dict(teams,[t1,t2,t3,t4])
    fixture_array = fixture(team_dict)
    result_df = pd.DataFrame(columns=["Week","HomeTeam","AwayTeam","FTHG","FTAG","Result"])
    

    for i in range(fixture_array.shape[0]):
        result_list = play_week(team_object_dict, df, fixture_array, i)
        
        input("Press ENTER to SIMULATE upcoming week")
        
        for j in range(fixture_array.shape[1]):
            if result_list[j][0] > result_list[j][1]:
                result = "HW"
            elif result_list[j][0] < result_list[j][1]:  
                result = "AW"
            else:
                result="TIE"
                
            c_dict = {
                "Week":i,
                "HomeTeam":team_dict[fixture_array[i][j][0]],
                "AwayTeam":team_dict[fixture_array[i][j][1]],
                "FTHG":result_list[j][0],
                "FTAG":result_list[j][1],
                "Result":result
            }
            
            df_c = pd.DataFrame([c_dict])
            result_df = result_df.append(df_c)
            
            result_df.reset_index(drop=True)
            
        updated_table = update_table(result_df)
        display(updated_table)
            
    return result_df
        

In [12]:
def update_table(result_df):
    standing_table = pd.DataFrame(index=teams,columns=["Games Played","Points","Win","Draw","Loss","Goal Difference", "Goal For" , "Goal Against"])
    standing_table.fillna(0,inplace=True)
    for idx, row in result_df.iterrows():
        home_team = row["HomeTeam"]
        away_team = row["AwayTeam"]
        result = row["Result"]
        
        standing_table.loc[home_team,"Games Played"] += 1
        standing_table.loc[away_team,"Games Played"] += 1

        goal_difference = abs(row["FTHG"] - row["FTAG"])
        h_goal = row["FTHG"]
        a_goal = row["FTAG"]
        
        if result=="HW":
            standing_table.loc[home_team,"Points"] += 3
            
            standing_table.loc[home_team,"Win"] += 1
            standing_table.loc[away_team,"Loss"] += 1
            
            standing_table.loc[home_team,"Goal For"] += h_goal
            standing_table.loc[away_team,"Goal For"] += a_goal
            
            standing_table.loc[home_team,"Goal Against"] += a_goal
            standing_table.loc[away_team,"Goal Against"] += h_goal
            
            standing_table.loc[home_team,"Goal Difference"] += goal_difference
            standing_table.loc[away_team,"Goal Difference"] -= goal_difference
            
        if result=="AW":
            standing_table.loc[away_team,"Points"] += 3
            
            standing_table.loc[home_team,"Loss"] += 1
            standing_table.loc[away_team,"Win"] += 1
            
            standing_table.loc[home_team,"Goal For"] += h_goal
            standing_table.loc[away_team,"Goal For"] += a_goal
            
            standing_table.loc[home_team,"Goal Against"] += a_goal
            standing_table.loc[away_team,"Goal Against"] += h_goal
            
            standing_table.loc[home_team,"Goal Difference"] -= goal_difference
            standing_table.loc[away_team,"Goal Difference"] += goal_difference
            
        if result=="TIE":
            standing_table.loc[home_team,"Points"] += 1
            standing_table.loc[away_team,"Points"] += 1
            
            standing_table.loc[home_team,"Draw"] += 1
            standing_table.loc[away_team,"Draw"] += 1
            
            standing_table.loc[home_team,"Goal For"] += h_goal
            standing_table.loc[away_team,"Goal For"] += a_goal
            
            standing_table.loc[home_team,"Goal Against"] += a_goal
            standing_table.loc[away_team,"Goal Against"] += h_goal
    
    standing_table.sort_values(by=["Points","Goal Difference"],axis=0,ascending=False,inplace=True)
    return standing_table


In [13]:
result_df = main(teams)

Press ENTER to SIMULATE upcoming week 


,Games Played,Points,Win,Draw,Loss,Goal Difference,Goal For,Goal Against
Man City,1,3,1,0,0,1,1,0
Arsenal,1,1,0,1,0,0,0,0
Liverpool,1,1,0,1,0,0,0,0
Chelsea,1,0,0,0,1,-1,0,1


Press ENTER to SIMULATE upcoming week 


,Games Played,Points,Win,Draw,Loss,Goal Difference,Goal For,Goal Against
Man City,2,6,2,0,0,3,4,1
Liverpool,2,2,0,2,0,0,1,1
Chelsea,2,1,0,1,1,-1,1,2
Arsenal,2,1,0,1,1,-2,1,3


Press ENTER to SIMULATE upcoming week 


...

,Games Played,Points,Win,Draw,Loss,Goal Difference,Goal For,Goal Against
Man City,4,9,3,0,1,4,10,6
Arsenal,4,7,2,1,1,0,6,6
Liverpool,4,3,0,3,1,-2,3,5
Chelsea,4,2,0,2,2,-2,2,4


Press ENTER to SIMULATE upcoming week 


,Games Played,Points,Win,Draw,Loss,Goal Difference,Goal For,Goal Against
Man City,5,12,4,0,1,5,14,9
Arsenal,5,10,3,1,1,1,8,7
Liverpool,5,3,0,3,2,-3,6,9
Chelsea,5,2,0,2,3,-3,3,6


Press ENTER to SIMULATE upcoming week 


,Games Played,Points,Win,Draw,Loss,Goal Difference,Goal For,Goal Against
Man City,6,13,4,1,1,5,15,10
Arsenal,6,10,3,1,2,-1,11,12
Liverpool,6,6,1,3,2,-1,11,12
Chelsea,6,3,0,3,3,-3,4,7
